## В следующих заданиях мы будем работать с базой данных, в которой имеется три таблицы:

1. Таблица с данными о пользователях (user):
    - id - уникальный идентификатор пользователя (primary key)
    - gender - пол
    - age - возраст
    - country - страна
    - city - город
    - exp_group - экспериментальная группа
    - os - операционная система
    - source - источник трафика
2. Таблица с данными о постах (post):
    - id - уникальный идентификатор поста (primary key)
    - text - текст поста
    - topic - тема поста
3. Таблица с данными о действиях пользователей (feed_action):
    - user_id (——>) user (id)- идентификатор пользователя     
    - post_id (——>) post (id)- идентификатор поста     
    - action - совершенное в сети действие     
    - time- время действия

In [1]:
import pandas as pd

df = pd.read_sql(
    """SELECT * FROM feed_action LIMIT 3""",
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df

,user_id,post_id,action,time
0,94620,6623,view,2022-01-18 18:02:13
1,125445,6636,view,2022-01-18 18:02:13
2,121623,6513,like,2022-01-18 18:02:13


Создам функцию, что бы проще отправлять запросы, которая будет возвращать датафрейм

In [2]:
def query(x):
    df = pd.read_sql(x,
                     con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
                         "postgres.lab.karpov.courses:6432/startml"
                    )
    return df

Проверю работоспособность

In [3]:
query("""
SELECT * 
FROM feed_action 
LIMIT 3
""")

,user_id,post_id,action,time
0,94620,6623,view,2022-01-18 18:02:13
1,125445,6636,view,2022-01-18 18:02:13
2,121623,6513,like,2022-01-18 18:02:13


### Задание 1

SELECT (1/2)

Сделайте запрос на все колонки из таблицы user. Для этого может понадобиться взять user в кавычки, вот так: "user" - чтобы PostgreSQL не спутал ее со своей встроенной таблицей user, где он ведет учет всех пользователей.

Напишите через пробел, с какого id начинаются записи и какой город у самого первого ID (на английском)

In [4]:
query("""
SELECT *
FROM "user"
LIMIT 1
""")

,id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads


Ответ: 200 Degtyarsk

### Задание 2

SELECT (2/2)

Отберите уникальные значения тем (topic) в таблице постов (post).

Напишите через пробел количество записей и самое длинное название топика.

In [5]:
query("""
SELECT DISTINCT topic
FROM post
""")

,topic
0,tech
1,covid
2,movie
3,entertainment
4,politics
5,sport
6,business


Ответ: 7 entertainment

### Задание 3

Фильтры (1/2)

Отберите пользователей старше 30 с устройством на iOS. Каков ID и город записи с минимальным ID (он идет первым в таблице)? 

Запишите значения через пробел.

In [6]:
query("""
SELECT id, city
FROM "user"
WHERE age > 30 AND os = 'iOS'
LIMIT 1
""")

,id,city
0,212,Podolsk


Ответ: 212 Podolsk

### Задание 4

Фильтры (2/2)

Отберите пользователей, которые не из России и при этом либо их экспериментальная группа не 0 и не 3 или их город – Минск.

Напишите через пробел первые 4 ID в такой выдаче.

In [7]:
query("""
SELECT id
FROM "user"
WHERE NOT(country = 'Russia')
AND (NOT (exp_group = 0 OR exp_group = 3)) OR city = 'Minsk'
LIMIT 4
""")

,id
0,211
1,213
2,216
3,218


Ответ: 211 213 216 218

### Задание 5

Группировка (1/5)

Подсчитайте средний возраст пользователей в разрезе страны. В ответ напишите средний возраст пользователей из Кипра. 

Ответ округлите до двух знаков после точки.

In [8]:
query("""
SELECT country, ROUND(AVG(age),2)
FROM "user"
GROUP BY country
""")

,country,round
0,Azerbaijan,26.78
1,Belarus,27.28
2,Cyprus,28.07
3,Estonia,28.15
4,Finland,27.13
5,Kazakhstan,27.22
6,Latvia,27.22
7,Russia,27.19
8,Switzerland,27.95
9,Turkey,27.45


Ответ: 28.07

### Задание 6

Группировка (2/5)

Сгруппируйте пользователей сначала по экспериментальной группе, а затем по операционной системе. В каждой группе подсчитайте количество пользователей, максимальный и минимальный возраст.

Сохраните результаты в файл CSV колонками **exp_group, os, total_users, max_age, min_age (обратите внимание на название последних трех колонок!)**, используя запятую в качестве сепаратора, и отправьте файл в LMS.

In [9]:
query("""
SELECT exp_group, os, 
COUNT(id) as total_users, 
MAX(age) as max_age, 
MIN(age) as min_age
FROM "user"
GROUP BY exp_group, os
""").to_csv("step_6.csv", sep=',', index=False)

### Задание 7

Группировка (3/5)

В каждой категории постов определите длину самого длинного текста, используя таблицу post. 

В ответ отправьте тему, у которой длина самого большого поста равна по порядку 25 000.

In [10]:
query("""
SELECT MAX(LENGTH(text)) AS max_len, topic
FROM post
GROUP BY topic
ORDER BY max_len DESC
""")

,max_len,topic
0,25392,politics
1,19004,entertainment
2,16063,tech
3,10234,movie
4,9438,sport
5,5378,business
6,148,covid


Ответ: politics

### Задание 8

Группировка (4/5)

Найдите страну с наименьшим числом пользователей среди стран, у которых общее число пользователей больше 1000. Введите ее название на английском.

In [11]:
query("""
SELECT COUNT(id) AS total_users, country
FROM "user"
GROUP BY country
HAVING COUNT(id) > 1000
ORDER BY total_users
LIMIT 1
""")

,total_users,country
0,1542,Azerbaijan


Ответ: Azerbaijan

### Задание 9

Группировка (5/5)

Отберите для пользователей из Москвы экспериментальные группы, в которых средний возраст больше 27.2. Напишите через пробел число пользователей в этих группах.

In [12]:
query("""
SELECT AVG(age) AS mean_age, COUNT(id) as total_count
FROM "user"
WHERE city = 'Moscow'
GROUP BY exp_group
HAVING AVG(age) > 27.2
""")

,mean_age,total_count
0,27.325102,4414
1,27.314299,4378


Ответ: 4414 4378

### Задание 11

Сортировка (1/2)

Сейчас мы будем работать с другой таблицей под названием post. Найдите в таблице post 3 темы с наибольшим количеством постов. Напишите через пробел названия этих тем.

In [13]:
query("""
SELECT topic, COUNT(id)
FROM post
GROUP BY topic
ORDER BY COUNT(id) DESC
LIMIT 3
""")

,topic,count
0,movie,3000
1,covid,1799
2,sport,510


Ответ: movie covid sport

### Задание 10

Сортировка (2/2)

Отберите пользователей из Воронежа (Voronezh) и отсортируйте их по возрасту в убывающем порядке. Для равных возрастов отсортируйте по возрастанию экспериментальной группы.

Напишите ID второго в порядке выдачи пользователя.

In [14]:
query("""
SELECT id, age, exp_group
FROM "user"
WHERE city = 'Voronezh'
ORDER BY age DESC, exp_group
LIMIT 2
""")

,id,age,exp_group
0,89990,71,0
1,2441,71,3


Ответ: 2441

### Задание 11

Join (1/2)

Отберите post_id, time, age и os пользователя для всех лайков пользователей из Омска. 

Отсортируйте результаты по убыванию времени действия и загрузите в LMS первые 100 значений в формате CSV (сепаратор "запятая", колонки с теми же названиями, что и в задании).

In [15]:
query("""
SELECT f.post_id, f.time, u.age, u.os
FROM "user" u
JOIN feed_action f ON f.user_id = u.id
WHERE u.city = 'Omsk' AND f.action = 'like'
ORDER BY f.time DESC
LIMIT 100
""").to_csv('step_11.csv', index=False)

### Задание 12

Join (2/2)

Найдите город, пользователи которого в возрасте 36 лет 2021-12-01 числа просмотрели запись на тему covid меньше всего раз относительно других городов. Введите его название на английском.

In [16]:
query("""
SELECT u.city, COUNT(f.action) AS view
FROM "user" u
JOIN feed_action f ON f.user_id = u.id
JOIN post p ON f.post_id = p.id
WHERE u.age = 36
AND f.action = 'view'
AND p.topic = 'covid'
AND DATE(f.time) = '2021-12-01'
GROUP BY u.city
ORDER BY COUNT(f.action)
LIMIT 1
""")

,city,view
0,Arkhangelsk,5


# Задание 13

## Топ постов

Напишите SQL запрос, используя который вы получите:
- топ N постов по лайкам,
- количество пролайкавших пользователей,
- время последнего лайка.

В качестве ответа напишите через пробел первые 3 ID постов из этого топа.

**Сохраните запрос, он может понадобиться в будущем в финальном проекте**

In [17]:
query("""
SELECT post_id, COUNT(action), MAX(time) 
FROM feed_action
WHERE action = 'like'
GROUP BY post_id
ORDER BY COUNT(action) DESC
LIMIT 3
""")

,post_id,count,max
0,4866,31714,2022-01-01 13:57:43
1,5127,29637,2022-01-05 13:52:17
2,6448,27939,2022-01-22 05:50:39


Ответ: 4866 5127 6448